In [6]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [8]:
!pip install gensim

In [2]:
import numpy as np
import tensorflow as tf

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/macos/Library/Python/3.6/lib/python/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
import gensim

model = gensim.models.KeyedVectors.load_word2vec_format('/Users/macos/Desktop/School/20172/Project 2/GoogleNews-vectors-negative300.bin', binary=True)

In [5]:
vocab = model.wv.vocab

In [6]:
def preprocess_sentence(sentence):
    marks = ['.', ',', '\n', '?', '!', '-', '(', ')', '/']
    for mark in marks:
        sentence = sentence.replace(mark, '')
    return sentence

In [7]:
def get_lines_from_file(file_path):
    file  = open(file_path, 'r')
    lines = file.readlines()
    
    for i in range(len(lines)):
        lines[i] = preprocess_sentence(lines[i])
    return lines

In [8]:
def get_sentence_vector(sentence):
    sentence_vector = []
    tokens = set(sentence.split(' '))
    for token in tokens:
        if token in vocab:
           sentence_vector.append(model[token])
    return sentence_vector

In [9]:
def get_train_data(file):
    train_data = []
    lines = get_lines_from_file(file)
    for line in lines:
        train_data.append(get_sentence_vector(line))
    return train_data

In [10]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "rnn"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [11]:
N_INPUTS = 300
CONTEXT_SIZE = 5
N_UNITS = 1024
N_CLASSES = 300
BATCH_SIZE = 32
N_RNN_LAYERS = 3

In [12]:
def get_sequence_length(X):
    sequence_length = []
    for s in range(np.shape(X)[0]):
        sequence_length = np.insert(arr=sequence_length, obj=s, values=int(np.shape(X[s])[0]), axis=0)
    return sequence_length

In [13]:
# path = './drive/test1.txt'
# train_data = get_train_data(path)
# test_path = './drive/input_test.txt'
# test_data = get_train_data(test_path)

path = '/Users/macos/Desktop/School/20172/Project 2/training_sentence.txt'
train_data = get_train_data(path)
# test_path = '/Users/macos/Desktop/School/20172/Project 2/test_sentence.txt'
test_path = '/Users/macos/Desktop/School/20172/Project 2/test/test_result.txt'
test_data = get_train_data(test_path)

In [14]:
MAX_LENGTH = np.max(get_sequence_length(train_data))

print(MAX_LENGTH)

45.0


In [15]:
def pad_zero_into_X(X):
    for i in range(len(X)):
        y = np.zeros(dtype=float, shape=(int(MAX_LENGTH-len(X[i])), N_INPUTS))
        X[i] = np.concatenate((X[i], y), axis=0)
    return X

In [16]:
def pad_zero_into_y(y):
    for i in range(len(y)):
        z = np.zeros(dtype=float, shape=(int(MAX_LENGTH-len(y[i]))))
        y[i] = np.concatenate((y[i], z), axis=0)
    return y

In [17]:
def length(sequence):
    used = tf.sign(tf.reduce_max(tf.abs(sequence), 2))
    length = tf.reduce_sum(used, 1)
    length = tf.cast(length, tf.int32)
    return length

In [18]:
def _weight_and_bias(in_size, out_size):
    weight = tf.truncated_normal([in_size, out_size], stddev=0.01)
    bias = tf.constant(0.1, shape=[out_size])
    return tf.Variable(weight), tf.Variable(bias)

In [19]:
def get_X_y_from_data(train_data):
    X = pad_zero_into_X(train_data)
    y = []
    for x in train_data:
        yi = [x[i+1] for i in range(np.shape(x)[0] - 1)]
        z = np.zeros(shape=(1, N_INPUTS))
        yi = np.concatenate((yi, z), axis=0)
        
        y.append(yi)
    return X, y    

In [20]:
print(np.shape(train_data[0]))
train_x, train_y = get_X_y_from_data(train_data)
test_x, test_y = get_X_y_from_data(test_data)

(12, 300)


In [21]:
def get_batch(i):
    X_batch = [train_x[j] for j in range(i*BATCH_SIZE, (i + 1) * BATCH_SIZE)]
    y_batch = [train_y[j] for j in range(i*BATCH_SIZE, (i + 1) * BATCH_SIZE)]
    return np.array(X_batch), np.array(y_batch)

In [22]:
def shuffle(X, y):
    X = np.array(X)
    y = np.array(y)
    a = [i for i in range(np.shape(X)[0])]
    np.random.shuffle(a)
    return X[a], y[a]

In [23]:
def get_max_index(arr):
    arr = np.reshape(arr, newshape=[-1, 4])
    m = np.max(arr, axis=1, keepdims=True)
    one_hot = arr // m
    result = np.argmax(one_hot, axis=1)
    return result

In [24]:
# build RNN like speech recognition

reset_graph()

X = tf.placeholder(tf.float32, shape=[None, MAX_LENGTH, N_INPUTS], name='X')
y = tf.placeholder(tf.float32, shape=[None, MAX_LENGTH, N_INPUTS], name='y')

keep_prob = tf.placeholder_with_default(1.0, shape=(), name='keep_prob')

# them placeholder vi tri cua tu de du doan tu khi test

lstm_cells = [tf.contrib.rnn.BasicLSTMCell(num_units=N_UNITS) for layer in range(N_RNN_LAYERS)]
lstm_cells_drop = [tf.contrib.rnn.DropoutWrapper(cell, input_keep_prob=keep_prob)
              for cell in lstm_cells]
multi_cell = tf.contrib.rnn.MultiRNNCell(lstm_cells_drop)

outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32, sequence_length=length(X))

#### predict
weight, bias = _weight_and_bias(N_UNITS, N_CLASSES)

# Flatten to apply same weights to all time steps.
outputs = tf.reshape(outputs, [-1, N_UNITS])

prediction = tf.matmul(outputs, weight) + bias
prediction = tf.reshape(prediction, [-1, tf.cast(MAX_LENGTH, tf.int32), N_CLASSES])

loss = tf.losses.mean_squared_error(y, prediction)


#### optimizer
optimizer = tf.train.AdamOptimizer(0.001)

training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [25]:
n_epochs = 30

with tf.Session() as sess:
    init.run()
    arr_test_loss = []
#     for epoch in range(n_epochs):
#         train_x, train_y = shuffle(train_x, train_y)
#         for iteration in range(np.shape(train_x)[0] // BATCH_SIZE):
#             #all vector in a sentence are in a batch, may be make it different by concatenate all sentences
#             X_batch, y_batch = get_batch(iteration)
            
#             sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            
#         train_loss_value = loss.eval(feed_dict={X: X_batch, y: y_batch})
#         print(epoch, " Train loss value:", train_loss_value)
#     for i in range(np.shape(test_x)[0]):
#         xi = np.reshape(test_x[i], newshape = [-1, 45, 300])
#         yi = np.reshape(test_y[i], newshape = [-1, 45, 300])
#         test_loss = loss.eval(feed_dict={X: xi, y: yi})
#         arr_test_loss.append(test_loss)

    saver.restore(sess, "/Users/macos/Desktop/School/20172/Project 2/model/rnn/test_model_rnn.ckpt")
    test_path = '/Users/macos/Desktop/School/20172/Project 2/test/test_result.txt'
    test_x, test_y = get_X_y_from_data(test_data)
    for i in range(np.shape(test_x)[0]):
        xi = np.reshape(test_x[i], newshape = [-1, 45, 300])
        yi = np.reshape(test_y[i], newshape = [-1, 45, 300])
        test_loss = loss.eval(feed_dict={X: xi, y: yi})
        arr_test_loss.append(test_loss)
    dict = {0: 'A', 1: 'B', 2: 'C', 3: 'D'}
    new_arr = 1 / np.array(arr_test_loss)
    predict_answer = np.array(get_max_index(new_arr))
    print(dict[predict_answer[0]])

INFO:tensorflow:Restoring parameters from /Users/macos/Desktop/School/20172/Project 2/model/rnn/test_model_rnn.ckpt


INFO:tensorflow:Restoring parameters from /Users/macos/Desktop/School/20172/Project 2/model/rnn/test_model_rnn.ckpt


NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for /Users/macos/Desktop/School/20172/Project 2/model/rnn/test_model_rnn.ckpt
	 [[Node: save/RestoreV2_2 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/RestoreV2_2/tensor_names, save/RestoreV2_2/shape_and_slices)]]

Caused by op 'save/RestoreV2_2', defined at:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_insta

In [33]:
print(test_x[0].reshape(-1, 45, 300))

[[[-0.01574707 -0.02832031  0.08349609 ...  0.00686646  0.06103516
   -0.1484375 ]
  [ 0.09912109 -0.09765625 -0.12353516 ... -0.17480469  0.04956055
    0.09863281]
  [ 0.109375    0.140625   -0.03173828 ...  0.00765991  0.12011719
   -0.1796875 ]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]]


In [0]:
arr_ans = np.load('./drive/arr_ans.npy')

In [0]:
new_arr = 1 / np.array(arr_test_loss)
predict_answer = np.array(get_max_index(new_arr))
# print(predict_answer)

In [36]:
print(np.sum(arr_ans == predict_answer) / np.shape(arr_ans)[0])

0.2663755458515284
